In [1]:
# Infer the gender of the authors
import analyze_genders

df = analyze_genders.dataframe()

In [2]:
# Group by conference and calculate the ratio of male to female authors
by_conf = df.groupby(df['conf']).sum()
by_conf['m/f'] = by_conf['male'] / by_conf['female']

by_conf

,year,male,female,unisex,unknown,m/f
conf,,,,,,
edbt,22165,548,206,27,130,2.660194
icde,22165,945,431,141,590,2.192575
kdd,22165,1279,532,185,765,2.404135
pvldb,22165,1266,420,95,525,3.014286
sigir,22165,1397,545,130,642,2.563303
sigmod,22165,1001,328,51,420,3.051829


In [3]:
# Group by year and calculate the ratio of male to female authors
by_year = df.groupby(df['year']).sum()
by_year['m/f'] = by_year['male'] / by_year['female']

by_year

,male,female,unisex,unknown,m/f
year,,,,,
2010,526,156,32,181,3.371795
2011,502,183,51,199,2.743169
2012,538,176,46,225,3.056818
2013,575,190,38,218,3.026316
2014,545,206,64,253,2.645631
2015,605,236,47,215,2.563559
2016,610,227,52,267,2.687225
2017,594,207,60,308,2.869565
2018,621,280,78,318,2.217857
